<a href="https://colab.research.google.com/github/JustinShawAcademy/data-science/blob/main/3_Calculations_with_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python and Data Analysis 3 - Calculations with Data

**Goal:** The goal of this project is to learn to calculate information from measured data.

**Description:** Data that is given to us is often considered *measured* - it is a real world observation that is recorded and put into a DataFrame. To turn this into information, we need to be able to use the data in calculations. This workshop covers how to create *calculated columns*, and how to calculate *summary statistics*.

## Importing Data

In [2]:
import os
import urllib.request



# GitHub raw base URL
base_url = "https://raw.githubusercontent.com/JustinShawAcademy/data-science/main/data/"
files = ["AAPL.csv", "AMZN.csv", "GOOG.csv", "MSFT.csv"]

# download each file into the Colab filesystem
for f in files:
    urllib.request.urlretrieve(base_url + f, f)

print("Files downloaded to Colab VM")


Files downloaded to Colab VM


## 3A: Calculated Columns

A calculated column is a column that is added to a DataFrame based on existing columns. In the following DataFrame, we have price information for Amazon's stock. It contains the columns `date`, `open`, `high`, `low`, `close`, and `volume`. We can create a calculated column in two ways.

In [3]:
import pandas as pd
amzn = pd.read_csv('AMZN.csv')
print(amzn.head())

         date    open    high     low   close    volume
0  1997-05-16  1.8650  1.9792  1.7083  1.7292  14700000
1  1997-05-19  1.7083  1.7708  1.6250  1.7083   6106800
2  1997-05-20  1.7292  1.7500  1.6358  1.6358   5467200
3  1997-05-21  1.6042  1.6458  1.3750  1.4275  18853200
4  1997-05-22  1.4375  1.4483  1.3125  1.3958  11776800


### Calculations Across Two Columns

Frequently, we want to be able to carry out mathematical operations between two or more columns. The syntax is quite intuitive. For example, we might want to keep track of the daily gains/losses. This can be calculated by subtracting the close price from the opening price: `open - close`.

In [4]:
amzn['daily_change'] = amzn['open'] - amzn['close']
print(amzn.head())

         date    open    high     low   close    volume  daily_change
0  1997-05-16  1.8650  1.9792  1.7083  1.7292  14700000        0.1358
1  1997-05-19  1.7083  1.7708  1.6250  1.7083   6106800        0.0000
2  1997-05-20  1.7292  1.7500  1.6358  1.6358   5467200        0.0934
3  1997-05-21  1.6042  1.6458  1.3750  1.4275  18853200        0.1767
4  1997-05-22  1.4375  1.4483  1.3125  1.3958  11776800        0.0417


A similar syntax can be used for other operations, including multiplication (`*`), division (`/`), addition (`+`) and exponents (`**`). As another example, perhaps we want an approximation of value of trades executed on a particular day by multiplying an average price by the volume. To get an estimate of average price, we will use `(open + high + low + close) / 4`. The final formula is `((open + high + low + close) / 4) * volume)`.

In [5]:
amzn = amzn.drop(columns=['daily_change']) # Removes the 'daily_change' column we created earlier
amzn['daily_value'] = ((amzn['open'] + amzn['high'] + amzn['low'] + amzn['close']) / 4) * amzn['volume']
print(amzn.head())

         date    open    high     low   close    volume  daily_value
0  1997-05-16  1.8650  1.9792  1.7083  1.7292  14700000  26760247.50
1  1997-05-19  1.7083  1.7708  1.6250  1.7083   6106800  10400491.08
2  1997-05-20  1.7292  1.7500  1.6358  1.6358   5467200   9226993.44
3  1997-05-21  1.6042  1.6458  1.3750  1.4275  18853200  28527248.25
4  1997-05-22  1.4375  1.4483  1.3125  1.3958  11776800  16470149.22


**Challenges**:
 - Calculate the difference between the `high` and `low` columns
 - Return the higher value between the `open` and `close` columns (hint: look into the `max` function)
 - Return the `close` price as a percentage of the original `close` price (useful when comparing the growth of different stocks)

In [6]:
print('High low difference:', amzn['high'] - amzn['low'], sep='\n', end='\n\n')

# Note that we have to do the max across the two columns via axis=1 instead of
# across all rows
print('Open close max:', amzn[['open', 'close']].max(axis=1), sep='\n', end='\n\n')

close_growth = amzn['close'].iloc[-1] / amzn['close'].iloc[0]
print('Close percentage growth:', f"{close_growth * 100:.2f}%")

High low difference:
0         0.2709
1         0.1458
2         0.1142
3         0.2708
4         0.1358
          ...   
5802     42.6600
5803    101.4300
5804     96.0900
5805    135.1500
5806    118.1300
Length: 5807, dtype: float64

Open close max:
0          1.8650
1          1.7083
2          1.7292
3          1.6042
4          1.4375
          ...    
5802    2524.0600
5803    2600.8600
5804    2647.4500
5805    2603.5000
5806    2601.2100
Length: 5807, dtype: float64

Close percentage growth: 147179.04%


### Operations on a Column

There is another way to carry out calculations on a column, but it can easily be used for other operations too. The `apply` function takes a column or entire DataFrame and applies a function to each item. This is convenient when the operation we want to perform for each item is quite complex. Below, we have a function `change_date` that takes a date in the form `YYYY-MM-DD` as a string, and outputs it in the form `Month Day, Year`.

In [7]:
def change_date(original_date):
    year = original_date[0:4]    # Get the first four characters in the string
    month = original_date[5:7]   # Get the month from the string
    months = ["January", "February", "March", "April", "May", "June", "July",
              "August", "September", "October", "November", "December"]
    month_name = months[int(month) - 1] # e.g. int('01') - 1 == 0 giving January
    day = original_date[-2:]    # Get the last two characters in the string

    return month_name + " " + day + ", " + year

print(change_date(amzn['date'][0]))

May 16, 1997


Because this operation is quite complex, we created a new function for it, and now just need to `apply` `change_date` to our `date` column.

In [8]:
amzn['date'] = amzn['date'].apply(change_date)
print(amzn.head())

           date    open    high     low   close    volume  daily_value
0  May 16, 1997  1.8650  1.9792  1.7083  1.7292  14700000  26760247.50
1  May 19, 1997  1.7083  1.7708  1.6250  1.7083   6106800  10400491.08
2  May 20, 1997  1.7292  1.7500  1.6358  1.6358   5467200   9226993.44
3  May 21, 1997  1.6042  1.6458  1.3750  1.4275  18853200  28527248.25
4  May 22, 1997  1.4375  1.4483  1.3125  1.3958  11776800  16470149.22


Ultimately, `apply` allows us to carry out more complex operations on a column, and *abstract* their functionality into helper functions.

**Challenge:** create a new `volume_estimate` column by apply a function to the `volume` column which replaces values greater than 10000000 with `'high'` and everything else with `'low'`.

In [9]:
def estimate_volume(volume):
  return 'high' if volume > 10000000 else 'low'

amzn['volume_estimate'] = amzn['volume'].apply(estimate_volume)
print(amzn)

               date       open       high        low      close    volume  \
0      May 16, 1997     1.8650     1.9792     1.7083     1.7292  14700000   
1      May 19, 1997     1.7083     1.7708     1.6250     1.7083   6106800   
2      May 20, 1997     1.7292     1.7500     1.6358     1.6358   5467200   
3      May 21, 1997     1.6042     1.6458     1.3750     1.4275  18853200   
4      May 22, 1997     1.4375     1.4483     1.3125     1.3958  11776800   
...             ...        ...        ...        ...        ...       ...   
5802  June 08, 2020  2500.2000  2530.0000  2487.3400  2524.0600   3957871   
5803  June 09, 2020  2529.4400  2626.4300  2525.0000  2600.8600   5175950   
5804  June 10, 2020  2645.0000  2722.3500  2626.2600  2647.4500   4913985   
5805  June 11, 2020  2603.5000  2671.3800  2536.2300  2557.9600   5753417   
5806  June 12, 2020  2601.2100  2621.4800  2503.3500  2545.0200   5436127   

       daily_value volume_estimate  
0     2.676025e+07            high  
1

## 3B: Summary Statistics

Previously, we carried out operations to fill each row in a new or existing column with a calculated value. Now we turn our attention to *summary statistics*. These aggregate calculations accross multiple rows within the same column. There are many different types of summary statistics, but common ones are:
 - `size`: Counts the number of rows in the given column
 - `count`: Counts the number of rows, excluding NaNs, in the given column
 - `sum`: Calculates the sum of the values in the given column
 - `min` and `max`: Calculates the minimum or maximum value in the given column
 - `mean`, `median`, and `mode`: Calculates the average value in the given column
 - `std`: Calculates the standard deviation in the given column
 - `describe`: Many statistics at once

 Lets look at a few examples:

Get the mean close price in the `amzn` DataFrame.

In [10]:
amzn = pd.read_csv('AMZN.csv')
mean_close = amzn['close'].mean()
print("Mean Close Price: " + str(mean_close))

Mean Close Price: 357.8692016015154


Find the highest and lowest close price.

In [11]:
highest_close = amzn['close'].max()
print("Highest Close Price: " + str(highest_close))

lowest_close = amzn['close'].min()
print("Lowest Close Price: " + str(lowest_close))

Highest Close Price: 2647.45
Lowest Close Price: 1.3958


Find the median value for both the `high` and `low` column. We can calculate summary statistics on more than one column by passing a list of columns.

In [12]:
median_value = amzn[['high', 'low']].median()
print(median_value)

high    84.47
low     81.00
dtype: float64


Describe the close prices.

In [13]:
described = amzn['close'].describe()
print(described)

count    5807.000000
mean      357.869202
std       553.538559
min         1.395800
25%        37.835000
50%        82.700000
75%       352.152000
max      2647.450000
Name: close, dtype: float64


**Challenge:** find the greatest difference in `high` and `low` prices on a given day. Think about the steps you need to perform, and whether the given DataFrame contains all the information we need.

In [14]:
(amzn['high'] - amzn['low']).max()

170.74

### Summary Statistics by Group

If we have data from multiple categories in the same DataFrame, we can split it into separate DataFrames and then calculate the summary statistics. Lets look at the DataFrame we create before, with stock prices for Microsoft, Amazon, Google, and Apple.

In [15]:
stock_names = ['MSFT', 'AAPL', 'AMZN', 'GOOG']

df = pd.DataFrame()
for stock_name in stock_names:
    stock_df = pd.read_csv(f'{stock_name}.csv')
    stock_df['name'] = stock_name
    df = pd.concat([df, stock_df], ignore_index=True)
print(df)

             date       open       high        low      close      volume  \
0      1986-03-13     0.0885     0.1016     0.0885     0.0972  1031788800   
1      1986-03-14     0.0972     0.1024     0.0972     0.1007   308160000   
2      1986-03-17     0.1007     0.1033     0.1007     0.1024   133171200   
3      1986-03-18     0.1024     0.1033     0.0990     0.0998    67766400   
4      1986-03-19     0.0998     0.1007     0.0972     0.0981    47894400   
...           ...        ...        ...        ...        ...         ...   
25960  2020-06-08  1422.3400  1447.9900  1422.3400  1446.6100     1401827   
25961  2020-06-09  1445.3600  1468.0000  1443.2100  1456.1600     1409249   
25962  2020-06-10  1459.5400  1474.2600  1456.2700  1465.8500     1521378   
25963  2020-06-11  1442.4800  1454.4700  1402.0000  1403.8400     1983710   
25964  2020-06-12  1428.4900  1437.0000  1386.0200  1413.1800     1946367   

       name  
0      MSFT  
1      MSFT  
2      MSFT  
3      MSFT  
4    

Combining our knowledge of the `groupby` function with our knowledge of summary statistics, we can do the following:

In [16]:
stocks = df.groupby('name')
for stock in stocks.groups.keys():
    stock_df = stocks.get_group(stock)
    avg_vol = stock_df['volume'].mean()
    print(stock + " Avg Trading Volume: " + str(avg_vol))

AAPL Avg Trading Volume: 85488428.64624962
AMZN Avg Trading Volume: 7512299.565696573
GOOG Avg Trading Volume: 1753946.9469648562
MSFT Avg Trading Volume: 60234781.231410705


Even more simply, we can do `grouped-object['name-of-col'].summary-statistic()`.

In [17]:
print(stocks['volume'].mean())

name
AAPL    8.548843e+07
AMZN    7.512300e+06
GOOG    1.753947e+06
MSFT    6.023478e+07
Name: volume, dtype: float64


The key takeaway is that Pandas allows us to easily calculate columns, operate on existing columns, and create summary statistics for columns and groups.

**Challenge:** Find the greatest average close price across all four stocks and print out the stock along with it's average close.

In [18]:
avg_close = stocks['close'].mean()
greatest_stock = avg_close.idxmax()
print(greatest_stock, avg_close[greatest_stock])

GOOG 900.2986702236421
